<ul class="breadcrumb">
  <li><a href="1.2.Multivariate_Linear_Mixed_Effects_Model.ipynb">Multivariate Basics</a></li>
  <li><a href="2.2.Multivariate_Linear_Additive_Genetic_Model.ipynb">Multivariate Additive Genetic Model</a></li> 
  <li><a href="3.2.Multivariate_Linear_Mixed_Effects_Model_with_Genomic_Data.ipynb">Multivariate Genomic Data</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Effects Model with Genomic Data</font> 
</div>

In [2]:
using DataFrames,CSV,JWAS
using JWAS: Datasets,misc

In [3]:
?runMCMC

search: runMCMC



```
runMCMC(mme,df;Pi=0.0,estimatePi=false,chain_length=1000,starting_value=false,printout_frequency=100,missing_phenotypes=false,constraint=false,methods="conventional (no markers)",output_samples_frequency::Int64 = 0)
```

Run MCMC (marker information included or not) with sampling of variance components.

  * available **methods** include "conventional (no markers)", "BayesC0", "BayesC", "BayesCC","BayesB".
  * **missing_phenotypes**
  * **Pi** for single-trait analyses is a number; **Pi** for multi-trait analyses is a dictionary such as `Pi=Dict([1.0; 1.0]=>0.7,[1.0; 0.0]=>0.1,[0.0; 1.0]=>0.1,[0.0; 0.0]=>0.1)`,

      * if Pi (Π) is not provided in multi-trait analysis, it will be generated assuming all markers have effects on all traits.
  * save MCMC samples every **output_samples_frequency** iterations
  * **starting_value** can be provided as a vector for all location parameteres except marker effects.
  * print out the monte carlo mean in REPL with **printout_frequency**
  * **constraint**=true if constrain residual covariances between traits to be zero.


<button type="button" class="btn btn-lg btn-primary">Data</button> 

In [4]:
phenofile = Datasets.dataset("testMT","phenotype.txt")
genofile  = Datasets.dataset("testMT","genotype.txt")
pedfile   = Datasets.dataset("testMT","pedigree.txt");

### phenotypes

In [5]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [6]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [7]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


In [8]:
data = CSV.read(phenofile,delim = ',',header=true,null="NA");

<div class="span5 alert alert-info">
 <font size="5" face="Georgia">I. Multiple Traits Analyses with Marker Information</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

### Genetic covariance matrix and residual covariance matrix

In [9]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

In [10]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [11]:
model1 = build_model(model_equations,R);

In [12]:
set_covariate(model1,"age");

In [13]:
add_genotypes(model1,genofile,G,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [14]:
Pi=Dict([1.0; 1.0]=>0.7,[1.0;0.0]=>0.1,[0.0,1.0]=>0.1,[0.0; 0.0]=>0.1)
out = runMCMC(model1,data,Pi=Pi,chain_length=5000,methods="BayesC",
estimatePi=true,output_samples_frequency=5);

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[10.9589 0.626223; 0.626223 1.09589].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          5
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.1,[1.0, 0.0]=>0.1,[1.0, 1.0]=>0.7,[0.0, 0.0]=>0.1)
estimatePi                                     true

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.0

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:03


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [15]:
keys(out)

Base.KeyIterator for a Dict{Any,Any} with 7 entries. Keys:
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "MCMC samples for residual covariance matrix"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [16]:
file1="MCMC_samples_for_marker_effects_BW.txt"
file2="MCMC_samples_for_marker_effects_CW.txt";

In [17]:
get_breeding_values(model1,file1,file2)

2-element Array{Any,1}:
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV      │ PEV     │
├─────┼────┼──────────┼─────────┤
│ 1   │ S1 │ -2.90923 │ 20.5328 │
│ 2   │ D1 │ 0.289624 │ 60.4025 │
│ 3   │ O1 │ 10.7198  │ 194.511 │
│ 4   │ O3 │ -8.10023 │ 59.9866 │            
 4×3 DataFrames.DataFrame
│ Row │ ID │ EBV       │ PEV      │
├─────┼────┼───────────┼──────────┤
│ 1   │ S1 │ -0.407811 │ 0.796295 │
│ 2   │ D1 │ 0.64458   │ 3.42457  │
│ 3   │ O1 │ 1.41096   │ 4.65713  │
│ 4   │ O3 │ -1.64773  │ 3.77901  │

In [18]:
samples4G=get_additive_genetic_variances(model1,file1,file2);

samples4R=out["MCMC samples for residual covariance matrix"];

samples4h2=get_heritability(reformat(samples4G),reformat(samples4R));

In [19]:
writedlm("out3.txt",samples4G)

In [20]:
samples_genetic_correlation=get_correlations(reformat(samples4G));

In [21]:
writedlm("out.G.txt",reformat(reformat(samples4G)))

In [22]:
out=readdlm("out.G.txt")

4×1000 Array{Float64,2}:
 250.56      188.191     247.121    …  20.5917    17.3427       1.66152 
   0.375972   -2.63645    -6.64268     -1.01599   -4.75509e-17  0.342272
   0.375972   -2.63645    -6.64268     -1.01599   -4.75509e-17  0.342272
   2.58449     0.651393    2.27528      0.280738   1.51643      2.44501 

In [23]:
reformat(out)

1000-element Array{Array{Float64,2},1}:
 [250.56 0.375972; 0.375972 2.58449]         
 [188.191 -2.63645; -2.63645 0.651393]       
 [247.121 -6.64268; -6.64268 2.27528]        
 [239.311 -6.57474; -6.57474 3.09645]        
 [312.573 0.896203; 0.896203 4.23299]        
 [514.013 9.27911; 9.27911 4.26298]          
 [595.109 42.7164; 42.7164 10.7273]          
 [542.042 4.66542; 4.66542 10.0555]          
 [510.18 -3.3889; -3.3889 7.41538]           
 [502.717 16.7167; 16.7167 5.13881]          
 [587.524 72.8189; 72.8189 11.3825]          
 [537.371 73.5715; 73.5715 12.7434]          
 [341.641 30.7309; 30.7309 3.54294]          
 ⋮                                           
 [61.765 7.61555; 7.61555 1.31842]           
 [24.3645 -3.26021; -3.26021 0.503113]       
 [5.51657 -0.355884; -0.355884 0.188208]     
 [1.37264 -0.609465; -0.609465 0.428967]     
 [1.05501 0.0; 0.0 0.0]                      
 [0.808812 0.0; 0.0 0.0]                     
 [5.00493 0.0; 0.0 0.0]                 

In [24]:
out10=reformat(samples4h2)

2×1000 Array{Float64,2}:
 0.674227  0.41205    0.768594  0.905624  …  0.0636437  0.0757336  0.00607171
 0.319772  0.0384976  0.363702  0.576998     0.0160809  0.192369   0.183894  

In [25]:
reformat(out,2)

1000-element Array{Array{Float64,2},1}:
 [250.56 0.375972; 0.375972 2.58449]         
 [188.191 -2.63645; -2.63645 0.651393]       
 [247.121 -6.64268; -6.64268 2.27528]        
 [239.311 -6.57474; -6.57474 3.09645]        
 [312.573 0.896203; 0.896203 4.23299]        
 [514.013 9.27911; 9.27911 4.26298]          
 [595.109 42.7164; 42.7164 10.7273]          
 [542.042 4.66542; 4.66542 10.0555]          
 [510.18 -3.3889; -3.3889 7.41538]           
 [502.717 16.7167; 16.7167 5.13881]          
 [587.524 72.8189; 72.8189 11.3825]          
 [537.371 73.5715; 73.5715 12.7434]          
 [341.641 30.7309; 30.7309 3.54294]          
 ⋮                                           
 [61.765 7.61555; 7.61555 1.31842]           
 [24.3645 -3.26021; -3.26021 0.503113]       
 [5.51657 -0.355884; -0.355884 0.188208]     
 [1.37264 -0.609465; -0.609465 0.428967]     
 [1.05501 0.0; 0.0 0.0]                      
 [0.808812 0.0; 0.0 0.0]                     
 [5.00493 0.0; 0.0 0.0]                 

In [26]:
#genetic correlation between trait 1 and trait 2
report(reformat(samples4G),index=[1,2]);

Summary Stats:
Mean:           18.626749
Minimum:        -38.685950
1st Quartile:   0.000000
Median:         2.199540
3rd Quartile:   32.997333
Maximum:        208.030269
Length:         1000
Type:           Float64
nothing


<div class="span5 alert alert-info">
 <font size="5" face="Georgia">II. Multiple Traits Analyses with Marker Effects and Polygenic Effects</font><br> 
</div>

<button type="button" class="btn btn-lg btn-primary">Build Model</button> 

In [27]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";
model2          = build_model(model_equations,R);

set_covariate(model2,"age");

get pedigree information from a file

In [28]:
ped=get_pedigree(pedfile);

calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00

Finished!


In [29]:
GA = G*0.1
set_random(model2,"Animal",ped,GA)

In [30]:
GM = G*0.9
add_genotypes(model2,genofile,GM,separator=',',header=true);

5 markers on 4 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Run Model</button> 

In [31]:
out2=runMCMC(model2,data,chain_length=5000,methods="BayesC");

The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 
[7.89041 0.394521; 0.394521 0.789041].


MCMC Information:

methods                                      BayesC
chain_length                                   5000
starting_value                                false
printout_frequency                             5001
output_samples_frequency                          0
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([0.0, 1.0]=>0.0,[1.0, 0.0]=>0.0,[1.0, 1.0]=>1.0,[0.0, 0.0]=>0.0)
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [32]:
keys(out2)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "Model frequency"
  "Posterior mean of residual covariance matrix"
  "Posterior mean of marker effects"
  "Posterior mean of marker effects covariance matrix"
  "Posterior mean of location parameters"

In [33]:
out2["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
 2.04859   0.109106
 0.109106  0.192032